# This File will step by step create proof of concept data for entry into the database via .csv export/import

In [11]:
#First let's import some stuff I'm going to need:
import random
import csv

#Now let's create some new patient IDs
patientnum=0
patientID = []
def createPatientIDs():
    patientnum = input("How many patients will we have?\n")
    for i in range(0,int(patientnum)):
        patientID.append(1000000 + i)
createPatientIDs()

        #### I no longer need the following section as I procedurely generate Practice ID's for my scenario
# Let's get some Practice ID's set up        
#practiceid=[]
#def createPracticeIDs():
#    practicenum = input("How many practices are participating?\n")
#    for n in range(0,int(practicenum)):
#        practiceid.append(6000+n)

#createPracticeIDs()

How many patients will we have?
100


### Now we need to make some housekeeping elements to contain our data. To do this I will create a class for Patients to store ID, age, ethnicity, and other details

In [17]:
class Patient:
    def __init__(self,idn,age,gender,ethnicity):
        self.idn     = idn     # given Identification number
        self.age     = age     # age of individual
        self.gender  = gender  # biological gender
        self.ethnicity    = ethnicity    # ethnicity of individual
        #self.control = control # 1 if control / 0 if not a control
class Case:
    def __init__(self,casenum,pid,practice_id,date):
        self.casenum=casenum
        self.pid=pid
        self.practice_id=practice_id
        self.date=date

### With a general organization for a patient setup, more structure is needed to create faux patients for this scenario.  A dictionary of ethnicitys for example, but also dictionaries for gender and carie location (or control) per individual to best create randomness
##### Note that Mandibular is the lower jaw while Maxillary is the upper jaw. Additionally Left and right is deteremined from the perspective of the patient. Dentists do use specific numbers (and letters for childhood teeth) but I am interested in the regions instead of the specific teeth.

In [14]:
genderDict = {0:"F",1:"M"}
ethnicityDict = {0:"White", 1: "African American", 2: "Hispanic/Latino", 3: "Native American", 4: "Pacific Islander", 
            5: "East Asian", 6: "Indian/Middle Eastern", 7: "Other"}
carieDict = {1: "Maxillary Right Molar",       2: "Maxillary Right Pre-Molar", 3: "Maxillary Right Canine", 
             4: "Maxillary Incisors",     5: "Maxillary Left Canine",       6: "Maxillary Left Pre-Molar",  7: "Maxillary Left Molar", 
             8: "Mandibular Right Molar", 9: "Mandibular Right Pre-Molar", 10: "Mandibular Right Canine",  11: "Mandibular Incisor",
            12: "Mandibular Left Canine",13: "Mandibular Left Pre-Molar",  14: "Mandibular Left Molar"}
#lets make the region table
with open('Region_Table.csv', 'w', newline='') as ccsv:
    cFieldNames = ["Region", "regionDescriptor"]
    cwriter = csv.DictWriter(ccsv, fieldnames=cFieldNames)
    cwriter.writeheader()
    for num in range(1,15): #for each region
        cwriter.writerow({"Region": num, "regionDescriptor": carieDict[num]}) #write the region id and it's string descriptor

### From here I will arbitrarily select a number of bacteria and viruses to be entered into a dictionary. For testing purposes, the bacteria will be selected from these selected names.

In [15]:
#Below will be two abbreviated dictionaries with a selection of bacteria and Viruses
bacteriaDict = {0: "Abiotrophia defectiva", 1: "Actinomyces dentalis",2: "Actinomyces gerencseriae", 3:"Actinomyces graevenitzii", 
                4:"Aggregatibacter segnis", 5:"Actinomyces johnsonii", 6:"Atopobium parvulum", 7:"Campylobacter gracilis", 
                8: "Actinomyces oris", 9: "Capnocytophaga gingivalis", 10: "Comamonas testosteroni", 11: "Gemella bergeri", 
                12: "Haemophilus aegyptius" , 13: "Kingella denitrificans", 14: "Kingella oralis", 15: "Lachnoanaerobaculum orale", 
                16: "Lachnoanaerobaculum saburreum", 17: "Lactobacillus salivarius", 18: "Neisseria bacilliformis", 19: "Neisseria oralis", 
                20: "Oribacterium asaccharolyticum", 21: "Peptidiphaga gingivicola", 22: "Peptostreptococcus anaerobius", 23: "Porphyromonas endodontalis", 
                24: "Porphyromonas gingivalis", 25: "Prevotella dentalis", 26: "Prevotella denticola", 27: "Staphylococcus epidermidis", 
                28: "Streptococcus mutans", 29: "Treponema denticola", 30: "Veillonella denticariosi"
               }
virusDict = {0: "HSV-1", 1: "HSV-2",2: "Varicella", 3:"Herpes zoster", 
             4:"Coxsackievirus A", 5:"Coxsackievirus B", 6:"Epstein-Barr virus", 7:"Rubeola", 
             8: "Paramyxovirus", 9: "Kaposi sarcoma-associated herpecirus", 10: "human cytomegalovirus", 11: "human herpesvirus 6", 
             12: "human herpesvirus 7"
            }

## Now it's time to put things together and create patients each with 14 regions that each have a list of bacteria names with proportion prevalence and a list of viral names with proportion prevalence.
### A note about controls:  In this scenario I want about half of my patients as controls but this will be denoted by the patient's casenum. Controls have no cavities

In [18]:
patientDict = {}
with open('Patient_Table.csv', 'w', newline='') as patientcsv:
    patientFieldNames = ["Patient ID", "Age", "Biological Gender", "ethnicity"]
    patientwriter = csv.DictWriter(patientcsv, fieldnames=patientFieldNames)
    patientwriter.writeheader()
    for pid in patientID: #for each patient
        patientDict[pid] = Patient(pid,                           #pid from the previously created list
                             random.randint(18,75),           #random age between 18 and 75 years old
                             genderDict[random.randint(0,1)], #random gender
                             ethnicityDict[random.randint(0,7)]   #random ethnicity
                             )
        #write this patient to the patient csv file
        patientwriter.writerow({"Patient ID": patientDict[pid].idn, 
                                "Age": patientDict[pid].age, 
                                "Biological Gender": patientDict[pid].gender, 
                                "ethnicity": patientDict[pid].ethnicity,})


        

## Creating some helper functions while also populating the Case_table

In [19]:
# Inserting small helper functions here to help prevent repeats

def createDate():  #Generates a date in String form
    day=str(random.randint(1,28))
    month=str(random.randint(1,12))
    year=str(random.randint(2020,2022))
    string = month+"/"+day+"/"+year
    return string

def entry(casenum,region): #uses casenum and region to create an entryid
    return (str(casenum)+"_"+str(region))

# Looking back on this, I could have done this more efficiently by creating the list and picking from it 
            # instead of repeatedly getting a random number and checking it
            # THIS IS INEFFICIENT
def selectElements(start,    #starting number of the range of numbers to pick from
                   end,      #ending number of the range
                   num       #number of elements requested
                   ):
    elements = []
    x=0
    a = 0
    while x < num:
        a=a+1
        number = random.randint(start,end)
        if number in elements: #if i have already picked this number, do a no-op
            b=1
        else:                  #if the number is new, add it to my list
            elements.append(number)
            x=x+1
    #print(a) #tells me how many runs were needed to get the final list
    return elements
#selectElements(1,10,10) #testing statement

In [27]:
#create Cases and write them to Case_Table
caseDict = {}
for pid in patientID:
    control = "C"
    experimental = "E"
    #this if else ensures half our cases are controls
    if pid%2 >0:
        casenum = control+str(pid-1000000)
    else:
        casenum = experimental+str(pid-1000000)
    caseDict[pid]=Case(casenum,pid,random.randint(6000,6030),createDate())
with open('Case_Table.csv','w',newline='') as casecsv:
    caseFieldNames=["casenum","pid","practice_id","date"]
    casewriter = csv.DictWriter(casecsv,fieldnames=caseFieldNames)
    casewriter.writeheader()
    for case in caseDict:
        casewriter.writerow({"casenum":caseDict[case].casenum,"pid":caseDict[case].pid,"practice_id":caseDict[case].practice_id,"date":caseDict[case].date})

### The patients have now been created. From here, I can begin creating assay "results". 
 #### Each patient will have 28 lists of 10 names and their associated proportions

In [25]:
with open('Bacteria_Data_Table.csv','w',newline='') as bactcsv:
    bactFieldNames = ["entryid","casenum", "Region",
                     "Bacteria 1 Name", "Bacteria 1 Prevalence",
                     "Bacteria 2 Name", "Bacteria 2 Prevalence",
                     "Bacteria 3 Name", "Bacteria 3 Prevalence",
                     "Bacteria 4 Name", "Bacteria 4 Prevalence",
                     "Bacteria 5 Name", "Bacteria 5 Prevalence",
                     "Bacteria 6 Name", "Bacteria 6 Prevalence",
                     "Bacteria 7 Name", "Bacteria 7 Prevalence",
                     "Bacteria 8 Name", "Bacteria 8 Prevalence",
                     "Bacteria 9 Name", "Bacteria 9 Prevalence",
                     "Bacteria 10 Name", "Bacteria 10 Prevalence"
                     ]
    bactwriter = csv.DictWriter(bactcsv, fieldnames=bactFieldNames)
    bactwriter.writeheader()
    #the header has been written now, time to put some 'real' simulated data into the model
    for case in caseDict:        #for each case
        for region in carieDict: #for each region
            #At this point I need to figure out some prevalance data.
            #I will generate 10 floats (up to 2 decimal points).
            #I then will sum up the values, to get a denominator to divide each value by. 
            #Each value will then be multiplied by 100 to get a final prevalence %.
            #And Finally the percentages will be sorted in descending order.
            values = []
            for i in range(0,10):
                values.append(round(random.uniform(5.00,50.00), 2))
            sumvals = sum(values)
            finalvals = []
            for v in values:
                finalvals.append(round((v/sumvals) *100,2))
            finalvals = sorted(finalvals,reverse=True)
            # The values have been determined but it's now time to figure out what bacteria is present
            # The dictionary 'bacteriaDict' has 31 bacteria species. I will use the helper function written above 
                    # to randomly select 10 of them without repeating.
            bacteriaNums = selectElements(0,30,10) # We get the 10 numbers
            bacteriaNames = []
            for bnums in bacteriaNums:
                bacteriaNames.append(bacteriaDict[bnums]) # Find the numbers in the dictionary and translate them to names
            # Now we write this region for this patient to the csv and move to the next patient/region
            bactwriter.writerow({"entryid": entry(caseDict[case].casenum,region),
                                 "casenum": caseDict[case].casenum, 
                                 "Region": region,
                                 "Bacteria 1 Name"       : bacteriaNames[0], 
                                 "Bacteria 1 Prevalence" : finalvals[0],
                                 "Bacteria 2 Name"       : bacteriaNames[1], 
                                 "Bacteria 2 Prevalence" : finalvals[1],
                                 "Bacteria 3 Name"       : bacteriaNames[2], 
                                 "Bacteria 3 Prevalence" : finalvals[2],
                                 "Bacteria 4 Name"       : bacteriaNames[3], 
                                 "Bacteria 4 Prevalence" : finalvals[3],
                                 "Bacteria 5 Name"       : bacteriaNames[4], 
                                 "Bacteria 5 Prevalence" : finalvals[4],
                                 "Bacteria 6 Name"       : bacteriaNames[5], 
                                 "Bacteria 6 Prevalence" : finalvals[5],
                                 "Bacteria 7 Name"       : bacteriaNames[6], 
                                 "Bacteria 7 Prevalence" : finalvals[6],
                                 "Bacteria 8 Name"       : bacteriaNames[7], 
                                 "Bacteria 8 Prevalence" : finalvals[7],
                                 "Bacteria 9 Name"       : bacteriaNames[8],  
                                 "Bacteria 9 Prevalence" : finalvals[8],
                                 "Bacteria 10 Name"      : bacteriaNames[9], 
                                 "Bacteria 10 Prevalence": finalvals[9]
                
            }) 

### The above writes the bacterial frequency csv. I now do the exact same thing but for the viruses. So below this is a copy and paste as the above code

In [26]:
with open('Virus_Data_Table.csv','w',newline='') as viralcsv:
    viralFieldNames = ["entryid", "casenum", "Region",
                     "Virus 1 Name", "Virus 1 Prevalence",
                     "Virus 2 Name", "Virus 2 Prevalence",
                     "Virus 3 Name", "Virus 3 Prevalence",
                     "Virus 4 Name", "Virus 4 Prevalence",
                     "Virus 5 Name", "Virus 5 Prevalence",
                     "Virus 6 Name", "Virus 6 Prevalence",
                     "Virus 7 Name", "Virus 7 Prevalence",
                     "Virus 8 Name", "Virus 8 Prevalence",
                     "Virus 9 Name", "Virus 9 Prevalence",
                     "Virus 10 Name", "Virus 10 Prevalence"
                     ]
    viralwriter = csv.DictWriter(viralcsv, fieldnames=viralFieldNames)
    viralwriter.writeheader()
    #the header has been written now, time to put some 'real' simulated data into the model
    for case in caseDict:        #for each patient
        for region in carieDict: #for each region
            #At this point I need to figure out some prevalance data.
            #I will generate 10 floats (up to 2 decimal points).
            #I then will sum up the values, to get a denominator to divide each value by. 
            #Each value will then be multiplied by 100 to get a final prevalence %.
            #And Finally the percentages will be sorted in descending order.
            values = []
            for i in range(0,10):
                values.append(round(random.uniform(5.00,50.00), 2))
            sumvals = sum(values)
            finalvals = []
            for v in values:
                finalvals.append(round((v/sumvals) *100,2))
            finalvals = sorted(finalvals,reverse=True)
            # The values have been determined but it's now time to figure out what virus is present
            # The dictionary 'virusDict' has 13 viral species. I will use the helper function written above 
                    # to randomly select 10 of them without repeating.
            viralNums = selectElements(0,12,10) # We get the 10 numbers
            viralNames = []
            for vnums in viralNums:
                viralNames.append(virusDict[vnums]) # Find the numbers in the dictionary and translate them to names
            # Now we write this region for this patient to the csv and move to the next patient/region
            viralwriter.writerow({ "entryid":entry(caseDict[case].casenum,region),
                                  "casenum": caseDict[case].casenum, 
                                 "Region": region,
                                 "Virus 1 Name"       : viralNames[0], 
                                 "Virus 1 Prevalence" : finalvals[0],
                                 "Virus 2 Name"       : viralNames[1], 
                                 "Virus 2 Prevalence" : finalvals[1],
                                 "Virus 3 Name"       : viralNames[2], 
                                 "Virus 3 Prevalence" : finalvals[2],
                                 "Virus 4 Name"       : viralNames[3], 
                                 "Virus 4 Prevalence" : finalvals[3],
                                 "Virus 5 Name"       : viralNames[4], 
                                 "Virus 5 Prevalence" : finalvals[4],
                                 "Virus 6 Name"       : viralNames[5], 
                                 "Virus 6 Prevalence" : finalvals[5],
                                 "Virus 7 Name"       : viralNames[6], 
                                 "Virus 7 Prevalence" : finalvals[6],
                                 "Virus 8 Name"       : viralNames[7], 
                                 "Virus 8 Prevalence" : finalvals[7],
                                 "Virus 9 Name"       : viralNames[8],  
                                 "Virus 9 Prevalence" : finalvals[8],
                                 "Virus 10 Name"      : viralNames[9], 
                                 "Virus 10 Prevalence": finalvals[9]
                
            }) 